In [1]:
# magic! (don't worry about this)
%load_ext autoreload
%autoreload 2
#%matplotlib inline

In [2]:
# let us import some useful things
from lib import *
from classifiers import *
#from graphs import *
from sklearn import tree
import numpy as np 
from time import time 
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale


In [3]:
# load the datasets as numpy arrays
data=np.asarray(np.loadtxt("../data/leukemia_ALL_AML_matrix.txt"))
labels=np.asarray(np.loadtxt("../data/leukemia_ALL_AML_labels.txt"))


In [15]:
[samples,genes]=(data.shape)
print "samples: "+str(samples)
print "genes (features): "+str(genes)
plt.hist(labels,bins="auto")
plt.title("Histogram of labels")
plt.show()

samples: 72
genes (features): 5147


In [29]:
#perform principle component analysis in 2d & 3d space -- this combines your features  
#so that they can be plotted in 2-D or 3-D space 
from mpl_toolkits.mplot3d import Axes3D
pca3 = PCA(n_components=3)
pca3.fit(data)
data3d = pca3.transform(data)
#data.shape
pca2=PCA(n_components=2)
pca2.fit(data)
data2d=pca2.transform(data)

In [30]:
#visualize the data in 3-d space 
fig = plt.figure(1, figsize=(4, 3))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, 1, 1], elev=48, azim=134)
ax.scatter(data3d[:, 0], data3d[:, 1], data3d[:, 2], c="blue", cmap=plt.cm.spectral)
plt.show()

In [33]:
#visualize the data in 2-d space 
fig = plt.figure(1, figsize=(4, 3))
plt.clf()
plt.scatter(data2d[:, 0], data2d[:, 1], c="blue", cmap=plt.cm.spectral)
plt.show()

In [36]:
#Set K= number of clusters you will generate. 
#We start with 2 clusters (K=2)
K=2

In [46]:
#K-means clustering: Step 1
#Randomly assign cluster means 

#get the range of your data, we will start with our 2-dimensional dataset. 
minvals=np.min(data2d,axis=0)
maxvals=np.max(data2d,axis=0)
print(minvals)
print(maxvals)

#randomly select your K centroids to fall in the range of the data 
centroids=[] 
for centroid in range(K):
    centroid_x=np.random.random(minvals[0],maxvals[0])
    centroid_y=np.random.random(minvals[1],maxvals[1])
    centroids.append(tuple([centroid_x,centroid_y]))
print str(centroids)

[-74864.78266752 -77588.20257567]
[ 85582.9945689   35434.82972541]


TypeError: random_sample() takes at most 1 positional argument (2 given)

In [27]:
np.random.seed(42)
n_samples, n_features = data.shape
n_digits = len(np.unique(labels))

sample_size = 72

print("n_digits: %d, \t n_samples %d, \t n_features %d"
      % (n_digits, n_samples, n_features))


print(79 * '_')
print('% 9s' % 'init'
      '    time  inertia    homo   compl  v-meas     ARI AMI  silhouette')

'''
def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('% 9s   %.2fs    %i   %.3f   %.3f   %.3f   %.3f   %.3f    %.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=sample_size)))
'''
def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('% 9s   %.2fs    %i   %.3f   %.3f   %.3f   %.3f    %.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_)))

bench_k_means(KMeans(init='k-means++', n_clusters=n_digits, n_init=10),
              name="k-means++", data=data)


#bench_k_means(KMeans(init='random', n_clusters=n_digits, n_init=10),
#              name="random", data=data)

# in this case the seeding of the centers is deterministic, hence we run the
# kmeans algorithm only once with n_init=1
#pca = PCA(n_components=n_digits).fit(data)
#bench_k_means(KMeans(init=pca.components_, n_clusters=n_digits, n_init=1),
#              name="PCA-based",
#              data=data)
print(79 * '_')

###############################################################################
# Visualize the results on PCA-reduced data

reduced_data = PCA(n_components=2).fit_transform(data)
kmeans = KMeans(init='k-means++', n_clusters=n_digits, n_init=10)
kmeans.fit(reduced_data)

# Step size of the mesh. Decrease to increase the quality of the VQ.
h = .02     # point in the mesh [x_min, m_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1)
plt.clf()
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap=plt.cm.Paired,
           aspect='auto', origin='lower')

plt.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=2)
# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='x', s=169, linewidths=3,
            color='w', zorder=10)
plt.title('K-means clustering on the digits dataset (PCA-reduced data)\n'
          'Centroids are marked with white cross')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.show()

n_digits: 2, 	 n_samples 72, 	 n_features 5147
_______________________________________________________________________________
init    time  inertia    homo   compl  v-meas     ARI AMI  silhouette
k-means++   0.08s    347187652523   0.012   0.012   0.012   0.027    0.001
_______________________________________________________________________________


MemoryError: 

In [20]:
data


array([[  8.80000000e+01,   1.50910000e+04,   3.11000000e+02, ...,
          3.60000000e+01,   1.91000000e+02,  -3.70000000e+01],
       [  2.83000000e+02,   1.10380000e+04,   1.34000000e+02, ...,
          1.10000000e+01,   7.60000000e+01,  -1.40000000e+01],
       [  3.09000000e+02,   1.66920000e+04,   3.78000000e+02, ...,
          4.10000000e+01,   2.28000000e+02,  -4.10000000e+01],
       ..., 
       [  1.46000000e+02,   1.56890000e+04,   3.02000000e+02, ...,
         -2.00000000e+00,   2.10000000e+02,   1.60000000e+01],
       [  4.31000000e+02,   4.15700000e+04,   2.35000000e+02, ...,
          0.00000000e+00,   2.84000000e+02,  -7.30000000e+01],
       [  9.00000000e+00,   3.95380000e+04,   1.01000000e+02, ...,
          2.00000000e+01,   3.79000000e+02,  -6.00000000e+01]])